# API Tests

In [312]:
import random, string

def gen_url():
    letters = string.ascii_lowercase
    protocols = ["http://", "https://", "ftp://"]
    level1s = [".com", ".ru", ".fr", ".uk"]
    domain = ''.join(random.choice(letters) for i in range(8))
    path = ''.join(random.choice(letters) for i in range(15))
    return random.choice(protocols) + domain + random.choice(level1s) + "/" + path

## 1. GRPC

### 1.1 Set up

In [313]:
!python -m grpc_tools.protoc -I../api/proto/ --python_out=. --pyi_out=. --grpc_python_out=. ../api/proto/links.proto

In [314]:
import grpc
import links_pb2_grpc

channel = grpc.insecure_channel('localhost:50051')
stub = links_pb2_grpc.LinkServiceStub(channel)

In [315]:
import links_pb2
import re

status_pattern = re.compile("StatusCode.([A-Z_]+)\n")

def extract_error(string):
    return status_pattern.search(string).group(1)

def create_link_grpc(url):
    try:
        request = links_pb2.CreateLinkRequest(url=str(url))
        result = stub.CreateLink(request)
        return result.alias
    except grpc.RpcError as e:
        return extract_error(str(e))
        

def get_link_grpc(alias):
    try:
        request = links_pb2.GetLinkRequest(alias=str(alias))
        result = stub.GetLink(request)
        return result.url
    except grpc.RpcError as e:
        return extract_error(str(e))

def grpc_must_already_exists(url):
    return create_link_grpc(url) == "ALREADY_EXISTS"

def grpc_must_not_found(alias):
    return get_link_grpc(url) == "NOT_FOUND"

def grpc_must_bad_input(url):
    return create_link_grpc(url) == "INVALID_ARGUMENT"

def grpc_must_ok():
    url = gen_url()
    alias = create_link_grpc(url)
    resp = get_link_grpc(alias)
    res = resp == url
    if not res:
        print(f"Wrong result for:\nurl: {url}\nalias: {alias}\ngot response: {resp}")
    return res

### 1.2 Run tests

In [316]:
print("Running normal tests...")
GRPC_DEFAULT_OK = True
for i in range(100):
    if not grpc_must_ok():
        GRPC_DEFAULT_OK = False
if GRPC_DEFAULT_OK:
    print("Pass")
else:
    print("Tests failed!!!")

print("Running error handling tests...")
GRPC_ERRHANDLING_OK = True
url = gen_url()
alias = create_link_grpc(url)
if grpc_must_already_exists(url):
    print("OK")
else:
    print(f"Could not get ERROR_EXISTS_ALREADY")
    GRPC_ERRHANDLING_OK = False

if grpc_must_not_found("0123456789"):
    print("OK")
else:
    print("Could not get NOT_FOUND_ERROR")
    GRPC_ERRHANDLING_OK = False

if grpc_must_bad_input("not-a-url-at-all"):
    print("OK")
else:
    print(f"Wrong! Should be INVALID_ARGUMENT")
    GRPC_ERRHANDLING_OK = False

Running normal tests...
Pass
Running error handling tests...
OK
OK
OK


## 2. REST API

### 2.1. Preapare

In [317]:
import urllib3 

http = urllib3.PoolManager()

In [318]:
import json
import requests

def create_link_rest(url):
    data = json.dumps({
        "url": url,
    })
    r = http.request(
        'POST',
        'http://localhost:8080/new',
        headers = {
            'Content-Type': 'application/json'
        },
        body = data
    )
    return r

def get_link(alias):
    request_url = f"http://localhost:8080/{alias}"
    r = requests.get(request_url, allow_redirects=False)
    return r

def http_must_ok():
    generated_url = gen_url()
    response = create_link_rest(generated_url)
    alias = str(response.json()['shortLink'][-10:])
    response = get_link(alias)
    if response.status_code == 302:
        got_url = response.headers.get("Location")
    else:
        got_url = response.json().get("url")
    if got_url != generated_url:
        print(f"Error! Want: {generated_url}, got: {got_url}")
        return False
    return True

### 2.2 Run tests

In [319]:
HTTP_DEFAULT_OK = True
print("Runnning default tests...")
for i in range(100):
    if not http_must_ok():
        HTTP_DEFAULT_OK = False
if not HTTP_DEFAULT_OK:
    print("Test Failed")
else:
    print("Pass")

Runnning default tests...
Pass


In [320]:
HTTP_ERRHANDLING_OK = True
print("Running errors tests...")

alias_404 = "0010010011"
response = get_link(alias_404)
if response.status_code == 404:
    print("OK")
else:
    HTTP_ERRHANDLING_OK = False
    print(f"Error! Expected code 404 but got {response.status_code}")

generated_url = gen_url()
response = create_link_rest(generated_url)
response = create_link_rest(generated_url)
if response.status == 409:
    print("OK")
else:
    print(f"Error! Expected code 409 but got {response.status}")
    HTTP_ERRHANDLING_OK = False

response = create_link_rest("not-a-url-at-all")
if response.status == 400:
    print("OK")
else:
    print(f"Error! Expected code 400 but got {response.status_code}")
    HTTP_ERRHANDLING_OK = False

Running errors tests...
OK
OK
OK


## 3. Both

In [321]:
MIXED_DEFAULT_OK = True
print("Running mixed default test...")
for i in range(100):
    if not http_must_ok() or not grpc_must_ok():
        MIXED_DEFAULT_OK = False
    if not (http_must_ok() and http_must_ok()) or not (grpc_must_ok() and grpc_must_ok()):
        MIXED_DEFAULT_OK = False
if MIXED_DEFAULT_OK:
    print("Pass")
else:
    print("Tests failed!!!")

Running mixed default test...
Pass


In [322]:
MIXED_ERRHANDLING_OK = True

generated_url = gen_url()
create_link_rest(generated_url)
result = grpc_must_already_exists(generated_url)
if result == True:
    print("OK")
else:
    MIXED_ERRHANDLING_OK = False
    print(f"Expected to get ALREADY_EXISTS error from GRPC but got {result}")

generated_url = gen_url()
create_link_grpc(generated_url)
response = create_link_rest(generated_url)
if response.status == 409:
    print("OK")
else:
    print(f"Error! Expected code 409 but got {response.status}")
    MIXED_ERRHANDLING_OK = False

OK
OK


# Results

In [323]:
if GRPC_DEFAULT_OK and GRPC_ERRHANDLING_OK and HTTP_DEFAULT_OK and HTTP_ERRHANDLING_OK and MIXED_DEFAULT_OK and MIXED_ERRHANDLING_OK:
    print("SUCCESS: All tests have passed")
else:
    print("FAIL: Some tests failed!!!")

SUCCESS: All tests have passed
